In [ ]:
#
# Initialization
#

import os
import sys

import ipynbname
from pathlib import Path

# Set notebook's src module path. Note that you may have to update your IDE's project settings to do the same for the
#  local library imports to work the same
MODULE_PATH = ipynbname.path().parent.parent.parent.parent
sys.path.append(str(MODULE_PATH))

# Keep paths consistent throughout notebook
os.chdir(MODULE_PATH)

# This should always be `./src`
print(f"Current working directory [{os.getcwd()}]")

# Place all local artifacts in a disposable, git-ignored directory
local_artifact_dir = Path(os.getcwd()).parent / "out"
local_artifact_dir.mkdir(parents=True, exist_ok=True)

# Autoreload imports at the beginning of cell execution.
#  https://ipython.org/ipython-doc/3/config/extensions/autoreload.html
%load_ext autoreload
%autoreload 2

In [ ]:
#
# Setup utils
#

from utils.logger_util import LoggerUtil
from utils.utils import Utils
from utils import config

LOGGER = LoggerUtil(config.MODEL_ID, local_artifact_dir / "logs")
UTILS = Utils(LOGGER)

UTILS.describe_env()

In [ ]:
#
# Initialization
#

from freqtrade.configuration import Configuration

from utils import config

# First prime the user_data_dir key. This will take priority when merged with config.json
freqtrade_config = Configuration({"user_data_dir": config.FREQTRADE_USER_DATA_DIR})
freqtrade_config = freqtrade_config.load_from_files([str(config.FREQTRADE_USER_DATA_DIR / "config.json")])
freqtrade_config["user_data_dir"] = config.FREQTRADE_USER_DATA_DIR

In [ ]:
#
# Load data based on the freqtrade config
#

from freqtrade.data.history import load_pair_history

data_dir = config.FREQTRADE_USER_DATA_DIR / "data" / "binance"
candles = load_pair_history(
    datadir=data_dir,
    timeframe=freqtrade_config["timeframe"],
    pair=config.CRYPTO_PAIR)

print(f"Loaded {str(len(candles))} rows of data for {config.CRYPTO_PAIR} from {data_dir}")

In [ ]:
#
# You can use this cell for discovery and experimentation
#

import data_processing.marshal_features as mf
df = mf.marshal_candle_metadata(candles, drop_date_column=True)
print(df["log_return_close"].describe())
print(df["log_return_close_2"].describe())
print(df["close"].head(10))
print(df["log_return_close"].head(10))
print(df["log_return_close_2"].head(10))

# import matplotlib.pyplot as plt
# # df_plot = df.drop(['sell', 'buy', \"volume\", \"open\", \"high\", \"low\", \"close\", \"hma\"], axis=1)
# df_plot2 = df[["log_return_close"]].tail(60).copy()
# ax = df_plot2.plot(figsize=(12, 5))
# plt.grid()
# plt.legend(df_plot2.columns)
# plt.show()

# ax = df.plot(kind='scatter', x="volume", y='close', figsize=(15, 15))
# [plt.axvline(x) for x in mf.NATURAL_VOLUME_BREAKS]

# # This takes some time to generate
# from pandas.plotting import scatter_matrix
# scatter_matrix(df_plot, alpha=0.2, figsize=(12, 12), diagonal='kde')

# df_plot[-499:].plot(figsize=(12, 5), linewidth=2)
# plt.grid()
# plt.legend(df_plot.columns)
# plt.show()

# ax = df_plot.plot(kind='scatter', x='volume_bin', y='pattern_detected', figsize=(15, 15))
# ax = df_plot.plot(kind='scatter', x='pattern_count', y='pattern_detected', figsize=(15, 15))
# ax = df_plot.plot(kind='scatter', x='volume', y='pattern_detected', figsize=(15, 1))


# for (columnName, columnData) in df.iteritems():
#     print(columnName)
#     print(columnData.isnull().sum())

# rows_with_nan = [index for index, row in df.iterrows() if row.isnull().any()]
# print(rows_with_nan)

# import numpy as np
# nan_indices = np.where(df['volume_bin'].isnull())[0]
# print(nan_indices)
#
# for idx in nan_indices:
#     print(df.iloc[idx]['volume'])
#     print(df.iloc[idx]['volume_bin'])
#     print("-------")

# candles_df = candles.tail(550).copy()
# candles_df = candles.copy()
# print(candles_df.head())

In [ ]:
#
# Load strategy and generate indicators and predictions
#

from freqtrade.resolvers import StrategyResolver

strategy = StrategyResolver.load_strategy(freqtrade_config)

strategy_metadata = {
    "pair": config.CRYPTO_PAIR,

    # Useful to turn off when df was already generated in this cell with the candle metadata
    "marshal_candle_metadata": False,

    # Useful to turn off when doing feature engineering before needing to predict
    "run_inference": True,
}

# Generate buy/sell signals using strategy
if strategy_metadata.get('marshal_candle_metadata', True):
    cached_df = strategy.analyze_ticker(candles_df, strategy_metadata)
    df = cached_df.copy()
else:
    df = cached_df.copy()
    df = df.drop(['sell', 'buy'], axis=1, errors='ignore')

    # Feel free to uncomment/comment this line out or even change the range. It is meant to reduce how many predictions
    # are made since they are computationally expensive
    df = df["2021-05-01 00:00:00":"2021-05-31 00:00:00"]

    df = strategy.analyze_ticker(df, strategy_metadata)

print(df[config.FREQTRADE_MAX_CONTEXT:config.FREQTRADE_MAX_CONTEXT + 10])

In [ ]:
#
# Reload and rerun strategy to later back test on generated indicators and predictions
#

# Reload Config
freqtrade_config = Configuration({"user_data_dir": config.FREQTRADE_USER_DATA_DIR})
freqtrade_config = freqtrade_config.load_from_files([str(config.FREQTRADE_USER_DATA_DIR / "config.json")])
freqtrade_config["user_data_dir"] = config.FREQTRADE_USER_DATA_DIR

# Reload strategy
strategy = StrategyResolver.load_strategy(freqtrade_config)

# Rerun buy and sell generation
df = df.drop(['sell', 'buy'], axis=1, errors='ignore')
df = strategy.advise_buy(df, strategy_metadata)
df = strategy.advise_sell(df, strategy_metadata)

In [ ]:
#
# Back test the strategy based on the indicators generated above
#

# iPython, which is used in jupyter notebooks, are not compatible with the async event loops of freqtrade.
# This is a workaround for that issue
import nest_asyncio
nest_asyncio.apply()

from datetime import datetime, timezone

import freqtrade.optimize.backtesting as ft_backtest
from freqtrade.data import history

print(f"Generated {df['buy'].sum()} buy signals")
print(f"Generated {df['sell'].sum()} sell signals")

backtest_start_time = datetime.now(timezone.utc)
backtesting = ft_backtest.Backtesting(freqtrade_config)

backtesting._set_strategy(strategy)
strategy_name = backtesting.strategy.get_strategy_name()

preprocessed_data = {config.CRYPTO_PAIR: df}
min_date, max_date = history.get_timerange(preprocessed_data)

results = backtesting.backtest(
    processed=preprocessed_data,
    start_date=min_date,
    end_date=max_date,
    max_open_trades=freqtrade_config['max_open_trades'],
    position_stacking=backtesting.config.get('position_stacking', False),
    enable_protections=backtesting.config.get('enable_protections', False),
)
backtest_end_time = datetime.now(timezone.utc)

results.update({
    'backtest_start_time': int(backtest_start_time.timestamp()),
    'backtest_end_time': int(backtest_end_time.timestamp()),
})
backtesting.all_results[strategy_name] = results

stats = ft_backtest.generate_backtest_stats(preprocessed_data, backtesting.all_results, min_date=min_date, max_date=max_date)
ft_backtest.show_backtest_results(backtesting.config, stats)

In [ ]:
#
# Load backtested trades as dataframe
#

import pandas as pd

# Replicate relevant parts of freqtrade.data.btanalysis.load_backtest_data
def load_backtest_data():
    data = stats['strategy'][strategy_name]['trades']
    trades_df = pd.DataFrame(data)
    if not df.empty:
        trades_df['open_date'] = pd.to_datetime(
            trades_df['open_date'],
            utc=True,
            infer_datetime_format=True
        )
        trades_df['close_date'] = pd.to_datetime(
            trades_df['close_date'],
            utc=True,
            infer_datetime_format=True
        )

        if 'profit_ratio' not in trades_df.columns:
            trades_df['profit_ratio'] = trades_df['profit_percent']
        trades_df = trades_df.sort_values("open_date").reset_index(drop=True)
    return trades_df

trades = load_backtest_data()

# Show value-counts per pair
trades.groupby("pair")["sell_reason"].value_counts()

In [ ]:
from freqtrade.data.btanalysis import analyze_trade_parallelism

# Analyze the above
parallel_trades = analyze_trade_parallelism(trades, '5m')
parallel_trades.plot()

In [ ]:
from freqtrade.plot.plotting import generate_candlestick_graph

# Filter trades to one pair
trades_red = trades.loc[trades['pair'] == config.CRYPTO_PAIR]

# Generate candlestick graph
graph = generate_candlestick_graph(
    pair=config.CRYPTO_PAIR,
    data=df,
    trades=trades_red,
    indicators1=['sma20', 'ema50', 'ema55'],
    indicators2=['rsi', 'macd', 'macdsignal', 'macdhist']
)

graph.show()
# graph.show(renderer="browser")

In [ ]:
# print(df["2021-05-20 16:00:00":"2021-05-20 18:00:00"])
# df.to_csv("../out/pred_cache.csv")